## Imports

In [1]:
import math
import numpy as np
import pandas
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import confusion_matrix, accuracy_score

# Preparando el dataset de entrenamiento

In [190]:
data_train = pandas.read_csv('data/perrosTrainAllComplete.csv')

data_train

,Mascota,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,1,Adulto,Mediano,Macho,Bicolor,Blanco,Negro,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
1,2,Cachorro,Mediano,Hembra,Bicolor,Blanco,Marron,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Caidas
2,3,Adulto,Mediano,Macho,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Largas,Paradas
3,4,Adulto,Mediano,Hembra,Liso,Dorado,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Paradas
4,5,Cachorro,Chico,Hembra,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,1910,Adulto,Chico,Macho,Bicolor,Dorado,Blanco,NaN,Corto,Marron oscuro,Mediano,Mediano,Mediano,Caidas
316,1919,Cachorro,Mediano,Macho,Tricolor,Negro,Blanco,Marron,Largo,Marron oscuro,Corto,Corto,Mediano,Caidas
317,1940,Cachorro,Mediano,Macho,Liso,Marron,NaN,NaN,Corto,Marron claro,Largo,Mediano,Cortas,Caidas
318,1950,Adulto,Chico,Hembra,Bicolor,Negro,Marron,NaN,Largo,Marron oscuro,Mediano,Corto,Largas,Caidas


### Obtenemos la cantidad de valores vacios

In [191]:
null_value_stats = data_train.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Color de pelaje 2    129
Color de pelaje 3    291
dtype: int64

### Lleno los nulos con strings vacios y quito la columna "Mascota"

In [192]:
data_train.fillna("NaN", inplace=True)
prepared_data_train = data_train.drop('Mascota', axis=1)
prepared_data_train

,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,Adulto,Mediano,Macho,Bicolor,Blanco,Negro,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
1,Cachorro,Mediano,Hembra,Bicolor,Blanco,Marron,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Caidas
2,Adulto,Mediano,Macho,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Largas,Paradas
3,Adulto,Mediano,Hembra,Liso,Dorado,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Paradas
4,Cachorro,Chico,Hembra,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Adulto,Chico,Macho,Bicolor,Dorado,Blanco,NaN,Corto,Marron oscuro,Mediano,Mediano,Mediano,Caidas
316,Cachorro,Mediano,Macho,Tricolor,Negro,Blanco,Marron,Largo,Marron oscuro,Corto,Corto,Mediano,Caidas
317,Cachorro,Mediano,Macho,Liso,Marron,NaN,NaN,Corto,Marron claro,Largo,Mediano,Cortas,Caidas
318,Adulto,Chico,Hembra,Bicolor,Negro,Marron,NaN,Largo,Marron oscuro,Mediano,Corto,Largas,Caidas


### Imprimo los tipos de cada columna

In [193]:
print(prepared_data_train.dtypes)


Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


### Obtengo la lista de etiquetas de las categorias

In [194]:
dataset_labels = prepared_data_train.columns.to_list()
dataset_labels

['Edad',
 'Tamaño',
 'Sexo',
 'Patron de pelaje',
 'Color de pelaje 1',
 'Color de pelaje 2',
 'Color de pelaje 3',
 'Largo de pelaje',
 'Color de ojos',
 'Largo de hocico',
 'Largo de cola',
 'Largo de orejas',
 'Tipo de orejas']

In [195]:
mascotas_train_ids = data_train.Mascota
y = data_train.Mascota
print(mascotas_train_ids)

0         1
1         2
2         3
3         4
4         5
       ... 
315    1910
316    1919
317    1940
318    1950
319    1954
Name: Mascota, Length: 320, dtype: int64


Obtengo las categorias en base a las que no son numericas, en este caso todas son categorias

In [196]:
categorical_features_indices = np.where(prepared_data_train.dtypes != np.float)[0]
categorical_features_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

# Preparando el dataset de testeo

In [292]:
dataset_test = pandas.read_csv('./data/perrosTestComplete.csv')
dataset_test.head()

null_value_stats = dataset_test.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

dataset_test.fillna("NaN", inplace=True)
prepared_dataset_test = dataset_test.drop('Mascota', axis=1)
prepared_dataset_test.head()

print(prepared_dataset_test.dtypes)

mascotas_test_ids = dataset_test.Mascota
mascotas_test_ids.head()

Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


0       6
1      13
2      64
3    1048
4      78
Name: Mascota, dtype: int64

In [306]:
dataset_test = pandas.read_csv('./data/perrosTestComplete.csv')
dataset_test.head()

,Mascota,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,1,Adulto,Mediano,Macho,Bicolor,Blanco,Negro,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
1,2,Cachorro,Mediano,Hembra,Bicolor,Blanco,Marron,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Caidas
2,3,Adulto,Mediano,Macho,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Largas,Paradas
3,4,Adulto,Mediano,Hembra,Liso,Dorado,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Paradas
4,5,Cachorro,Chico,Hembra,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas


In [307]:
null_value_stats = dataset_test.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Color de pelaje 2    24
Color de pelaje 3    46
dtype: int64

In [308]:
dataset_test.fillna("NaN", inplace=True)
prepared_dataset_test = dataset_test.drop('Mascota', axis=1)
prepared_dataset_test.head()

,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,Adulto,Mediano,Macho,Bicolor,Blanco,Negro,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
1,Cachorro,Mediano,Hembra,Bicolor,Blanco,Marron,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Caidas
2,Adulto,Mediano,Macho,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Largas,Paradas
3,Adulto,Mediano,Hembra,Liso,Dorado,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Paradas
4,Cachorro,Chico,Hembra,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas


In [309]:
print(prepared_dataset_test.dtypes)

Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


In [310]:
mascotas_test_ids = dataset_test.Mascota
mascotas_test_ids.head()

0    1
1    2
2    3
3    4
4    5
Name: Mascota, dtype: int64

# Creando y entrenando el modelo

### Creacion del modelo

In [333]:
train_dataset = Pool(data=prepared_data_train,
                     label=mascotas_train_ids,
                        cat_features=categorical_features_indices)

# Initialize CatBoostClassifier
''' model = CatBoostClassifier(iterations=100,
                           depth=3,
                           loss_function="MultiLogloss") '''

model = CatBoostClassifier(iterations=3000,
                           learning_rate=0.15,
                           depth=3,
                           auto_class_weights="Balanced",
                           loss_function="MultiClass")


### Cargando un model

In [242]:
# model2 = CatBoostClassifier().load_model("models/model1.cbm", format='cbm')
# model3 = CatBoostClassifier().load_model(
#     "models/modelDepth3.cbm", format='cbm')
# model4 = CatBoostClassifier().load_model(
#     "models/modelDepth3Ite100.cbm", format='cbm')

model_depth3 = CatBoostClassifier().load_model(
    "models/modelDepth3.cbm", format='cbm')

model_100it_D10 = CatBoostClassifier().load_model(
    "models/modelDepth10Ite100.cbm", format='cbm')

model_100it_lr05_D8 = CatBoostClassifier().load_model(
    "models/modelIt100Lr05D8.cbm", format='cbm')

model_2000it_lr015_D3 = CatBoostClassifier().load_model(
    "models/modelIt2000Lr015D3.cbm", format='cbm')

model_MultiClassOneVsAll_100it_D3 = CatBoostClassifier().load_model(
    "models/MultiClassOneVsAll/modelIt100D3.cbm", format='cbm')

model_Depth6Ite1000 = CatBoostClassifier().load_model(
    "models/modelDepth6Ite1000.cbm", format='cbm')


# **Entrenamiento**

In [334]:
model.fit(train_dataset)

''' model.save_model("models/modelIt100D3.cbm",
                    format="cbm",
                    export_parameters=None,
                    pool=None) '''

''' model.save_model("models/modelIt2000Lr015D3.json",
                    format="json",
                    export_parameters=None,
                    pool=None) '''


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 5.7646205	total: 22.7s	remaining: 18h 56m 6s
1:	learn: 5.7466975	total: 41.3s	remaining: 17h 13m 1s
2:	learn: 5.7177882	total: 1m	remaining: 16h 54m 23s


KeyboardInterrupt: 

# Guardando el model

In [117]:
model.save_model("models/modelDepth3Ite100.cbm",
                    format="cbm",
                    export_parameters=None,
                    pool=None)

model.save_model("models/modelDepth3Ite100.json",
                    format="json",
                    export_parameters=None,
                    pool=None)


CatBoostError: There is no trained model to use save_model(). Use fit() to train model. Then use this method.

# Generar predicciones

In [312]:
eval_dataset = Pool(data=prepared_dataset_test,
                    label=mascotas_test_ids,
                    cat_features=categorical_features_indices)


In [313]:
# Get predicted classes
# preds_class = model.predict(eval_dataset)
# preds_class2 = model_depth3.predict(eval_dataset)
# preds_class3 = model3.predict(eval_dataset)
# preds_class4 = model4.predict(eval_dataset)
# preds_class_100i_10d = model_100it_10d.predict(eval_dataset)
# preds_class_2000i_3d = model_2000it_3d.predict(eval_dataset)
# preds_class_multi_100 = model.predict(eval_dataset)

# Get predicted probabilities for each class
# preds_proba = model.predict_proba(eval_dataset)
preds_proba_model_depth3 = model_depth3.predict_proba(eval_dataset)

preds_proba_model_100it_D10 = model_100it_D10.predict_proba(eval_dataset)

preds_proba_model_100it_lr05_D8 = model_100it_lr05_D8.predict_proba(eval_dataset)

preds_proba_model_2000it_lr015_D3 = model_2000it_lr015_D3.predict_proba(
    eval_dataset)

preds_proba_model_MultiClassOneVsAll_100it_D3 = model_MultiClassOneVsAll_100it_D3.predict_proba(
    eval_dataset)

preds_proba_modelDepth6Ite1000 = model_Depth6Ite1000.predict_proba(
    eval_dataset)


In [101]:
''' print("1: ",preds_class)
print("2: ",preds_class2)
print("3: ",preds_class3)
print("4: ",preds_class4)
print("5: ",preds_class_100i_10d) '''

' print("1: ",preds_class)\nprint("2: ",preds_class2)\nprint("3: ",preds_class3)\nprint("4: ",preds_class4)\nprint("5: ",preds_class_100i_10d) '

In [245]:
print(len(preds_proba_model_depth3))

5


## Probabilidades
Un array por cada mascota de testeo

In [36]:
# print(preds_proba[0])
# print(preds_proba2[0])
# print(preds_proba3[0])
# print(preds_proba4[0])
print(preds_proba_2000i_3d[0])

[6.26140509e-04 1.69331875e-04 4.03383182e-04 1.14481702e-04
 2.38157270e-03 6.06536351e-04 9.27039312e-05 1.57059303e-04
 1.04281290e-04 1.15267865e-04 2.89913448e-04 4.79095100e-04
 3.03035637e-04 1.58926783e-04 5.16031867e-05 1.25123122e-04
 4.51938462e-03 3.42525064e-04 3.30321159e-04 1.05464628e-04
 2.02569538e-04 6.22343097e-05 8.70825916e-04 2.48598575e-02
 3.47441732e-04 1.64427712e-02 1.78428490e-03 7.33432121e-05
 1.27363457e-03 5.05813639e-03 2.91240231e-03 1.37676113e-04
 2.44360820e-04 7.39147673e-05 7.22971088e-04 5.99907598e-05
 3.50249010e-04 1.75633165e-04 3.02062239e-04 2.93627393e-03
 2.41822542e-04 3.92227001e-04 1.67515060e-04 1.47005780e-03
 1.32848050e-03 9.32283694e-05 9.53372801e-05 1.01569044e-03
 1.76356331e-03 8.41098752e-04 3.06966075e-04 2.29711042e-04
 2.07694115e-04 1.80506664e-04 2.03295538e-03 2.80723568e-03
 7.99760937e-05 1.07930930e-02 2.26764655e-04 1.22160712e-04
 1.75744995e-04 2.03827488e-04 6.02994947e-04 1.64438087e-02
 1.05469912e-04 2.844797

In [90]:
''' contador = 0
while contador < 320:
    if(preds_proba_100i_10d[0][contador] == 0.031220734475029422):
        print(contador, " --> ", preds_proba_100i_10d[0][contador])
    contador = contador +1

print(preds_proba_100i_10d[0][116]) '''

id_mascota = np.where(data_train.Mascota == 3789)[0]

print(preds_proba_100i_10d[0][id_mascota])


[0.00375544]


### Supuestas predicciones

In [23]:
mascota_numero = 0

for prediccion in preds_class:
    probabilidad = math.trunc((preds_proba[mascota_numero][prediccion.index()])*100)
    print("-- Mascota "+str(mascotas_test_ids[mascota_numero])+" es similar a la mascota "+str(prediccion.item())
        +" => "+str(probabilidad)+"% de probabilidad")
    # print(str(preds_proba[mascota_numero]))
    print(dataset_test.loc[mascota_numero:mascota_numero, "Edad":"Tipo de orejas"].values)
    print(prepared_data_train.loc[prediccion.item():prediccion.item(), "Edad":"Tipo de orejas"].values)
    mascota_numero+=1


AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [55]:
class Mascota:
    def __init__(self,index, id, prob):
        self.id = id
        self.index = index
        self.prob = prob

    def to_string(self):
        return "{}\t|\t{}\t=>\t{}%".format(self.index,self.id,self.prob)

    def get_id(self):
        return self.id
    
    def get_prob(self):
        return self.prob

    def get_index(self):
        return self.index

# KeyValue

In [326]:
def keyValue(list_proba):
    mascota_test_index = 46
    # mascota_probs = list_proba[0]

    mascota_test_id = str(mascotas_test_ids[mascota_test_index].item())
    print(mascota_test_id)
    
    mascotas_probs = []

    for prob_index in range(0, len(list_proba[mascota_test_index])):
        probabilidad = list_proba[mascota_test_index][prob_index]*100
        # probabilidad = list_proba[mascota_test_index][prob_index]

        # print(probabilidad)
        # print(mascotas_train_ids[prob_index], " --> ",
            #   list_proba[0][prob_index], " >> ", probabilidad)
        # print(list_proba[mascota_test_index][41])
        # print(mascotas_train_ids[prob_index],"=>","{0:.2f}".format(probabilidad),"%")

        mascotas_probs.append(
            Mascota(prob_index,mascotas_train_ids[prob_index], probabilidad))
        
    # print(len(mascotas_probs))

    def orderProb(n):
        return n.get_prob()

    def orderId(n):
        return n.get_id()

    def orderIndex(n):
        return n.get_index()

    mascotas_probs.sort(key=orderProb, reverse=True)

    for mascota in mascotas_probs:
        print(mascota.to_string())
    # print(mascotas_probs[0].to_string())

# keyValue(preds_proba_model_depth3)
# keyValue(preds_proba_model_100it_D10)
# keyValue(preds_proba_model_100it_lr05_D8)
# keyValue(preds_proba_model_2000it_lr015_D3)
# keyValue(preds_proba_model_MultiClassOneVsAll_100it_D3)
keyValue(preds_proba_modelDepth6Ite1000)

47
86	|	1083	=>	10.922009771611613%
307	|	1863	=>	7.213908192117542%
0	|	1	=>	4.7205817716681695%
137	|	3950	=>	2.5415951222166573%
17	|	18	=>	2.4957096019144807%
25	|	26	=>	2.4544785334919084%
46	|	47	=>	2.0207059637697804%
306	|	1851	=>	1.8313799337834749%
239	|	1531	=>	1.597734315388236%
218	|	4397	=>	1.480548383016509%
201	|	4352	=>	1.4271877870278677%
40	|	41	=>	1.4156059917803963%
29	|	30	=>	1.411805820925359%
106	|	3601	=>	1.202947074231434%
68	|	1036	=>	1.1078049165223232%
287	|	1702	=>	1.0062274946211454%
34	|	35	=>	1.0038632649594723%
160	|	4210	=>	0.8952992674852769%
304	|	1838	=>	0.8710022136141428%
23	|	24	=>	0.8379549321602375%
109	|	3701	=>	0.8224146752765122%
49	|	50	=>	0.7855098972297646%
225	|	6045	=>	0.7515243624916941%
5	|	6	=>	0.7301684833817198%
26	|	27	=>	0.7296560701146195%
98	|	3528	=>	0.6920120786691194%
274	|	4535	=>	0.6194250999118993%
126	|	1219	=>	0.603002517386804%
44	|	45	=>	0.6006557848547335%
41	|	42	=>	0.5995285120159854%
39	|	40	=>	0.5942575803443089

In [ ]:
# 315 --> 0.0027274134200535954
